<a href="https://colab.research.google.com/github/DavidPachis/dataScience/blob/main/dataScience/Bono/data_qualityDavidVasquez.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/datascience-uniandes/data-quality-tutorial/blob/master/data-quality-tutorial.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data Quality tutorial

MINE-4101: Applied Data Science  
Univerisdad de los Andes  
  
Dataset: Homicides Colombia  
Source: [datos.gov.co](datos.gov.co)
  
Last update: September, 2022

## Instalaciones e Importaciones


In [6]:
!pip install pylev
!pip install wget

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=dfe71fe9a7ec8ef728963e6512d3719528995235bc528a465193c82689ab7e55
  Stored in directory: /root/.cache/pip/wheels/a1/b6/7c/0e63e34eb06634181c63adacca38b79ff8f35c37e3c13e3c02
Successfully built wget


In [1]:
# Importing required libraries
import wget
import re
from random import randint
from datetime import datetime

import numpy as np
import pandas as pd

import pylev

In [2]:
# Parameter for showing all columns when printing a datafr
pd.set_option('display.max_columns', None)

## lectura de dataset


In [490]:
# Loading data

homicides_df = pd.read_csv("https://raw.githubusercontent.com/DavidPachis/dataScience/main/Bono/homicides.csv")

In [491]:
# Printing the dataset dimensions
homicides_df.shape

(12400, 22)

In [492]:
# Printing column data types
homicides_df.dtypes

FECHA                 object
DEPARTAMENTO          object
MUNICIPIO             object
DIA                   object
HORA                  object
BARRIO                object
ZONA                  object
CLASE DE SITIO        object
ARMA O MEDIO          object
MOVIL VICTIMA         object
MOVIL AGRESOR         object
EDAD                 float64
GENERO                object
ESTADO CIVIL          object
CLASE EMPLEADO        object
PROFESION             object
ESCOLARIDAD           object
PAIS NACE             object
CODIGO DANE          float64
AÑO DE NACIMIENTO    float64
CÉDULA                object
CORREO                object
dtype: object

In [493]:
homicides_df

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
0,01/01/2021 12:00:00 AM,ANTIOQUIA,AMAGÁ,Jueves,6:00,EL VOLCAN,RURAL,TIENDA,ARMA BLANCA,A PIE,A PIE,44.0,MASCULINO,CASADO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,5030000.0,1971.0,42-908,agbnqg2122@unidatos.edu.co
1,01/01/2021 12:00:00 AM,ANTIOQUIA,BARBOSA,Jueves,9:00,VDA. MATASANOS,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,30.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,NO REPORTADO,5079000.0,1985.0,15-183,rbkeui3584@gmail.com
2,01/01/2021 12:00:00 AM,ANTIOQUIA,EL BAGRE,Jueves,19:00,PUERTO CLAVER,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,33.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5250000.0,1982.0,84-786,aorkhf9155@unidatos.edu.co
3,01/01/2021 12:00:00 AM,ANTIOQUIA,JARDÍN,Jueves,11:20,CRISTIANIA,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,40.0,MASCULINO,CASADO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5364000.0,1975.0,31-289,dhtemr6623@unidatos.edu.co
4,01/01/2021 12:00:00 AM,ANTIOQUIA,MEDELLÍN (CT),Juees,15:00,PICACHITO CNO REPORTADO6,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,CONTUNDENTES,A PIE,A PIE,66.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1949.0,66-363,artatj9268@unidatos.edu.co
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12395,12/31/2021 12:00:00 AM,VALLE,LA CUMBRE,Jueves,19:04,CGTO BITACO E27,RURAL,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,58.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,76377000.0,1957.0,39-442,correo4352@colombia.gov.co
12396,12/31/2021 12:00:00 AM,VALLE,PALMIRA,Jueves,22:35,LORETO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,18.0,FEMENINO,SOLTERO,ESTUDIANTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76520000.0,1997.0,25-368,fktatq9245@unidatos.edu.co
12397,12/31/2021 12:00:00 AM,VALLE,ZARZAL,Jueves,14:30,URB. EL ZARZALITO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,27.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76895000.0,1988.0,62-735,gafkup6393@gmail.com
12398,12/31/2021 12:00:00 AM,VICHADA,CUMARIBO,Jueves,22:55,EL CENTRO,URBANA,HOSPITALES,ARMA BLANCA,A PIE,A PIE,23.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,99773000.0,1992.0,47-126,correo736@unidatos.edu.co


## Borrar columna de total que estorba


In [494]:
homicides_df.drop(homicides_df.tail(1).index,inplace=True)

In [495]:
homicides_df

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
0,01/01/2021 12:00:00 AM,ANTIOQUIA,AMAGÁ,Jueves,6:00,EL VOLCAN,RURAL,TIENDA,ARMA BLANCA,A PIE,A PIE,44.0,MASCULINO,CASADO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,5030000.0,1971.0,42-908,agbnqg2122@unidatos.edu.co
1,01/01/2021 12:00:00 AM,ANTIOQUIA,BARBOSA,Jueves,9:00,VDA. MATASANOS,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,30.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,NO REPORTADO,5079000.0,1985.0,15-183,rbkeui3584@gmail.com
2,01/01/2021 12:00:00 AM,ANTIOQUIA,EL BAGRE,Jueves,19:00,PUERTO CLAVER,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,33.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5250000.0,1982.0,84-786,aorkhf9155@unidatos.edu.co
3,01/01/2021 12:00:00 AM,ANTIOQUIA,JARDÍN,Jueves,11:20,CRISTIANIA,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,40.0,MASCULINO,CASADO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5364000.0,1975.0,31-289,dhtemr6623@unidatos.edu.co
4,01/01/2021 12:00:00 AM,ANTIOQUIA,MEDELLÍN (CT),Juees,15:00,PICACHITO CNO REPORTADO6,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,CONTUNDENTES,A PIE,A PIE,66.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1949.0,66-363,artatj9268@unidatos.edu.co
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12394,12/31/2021 12:00:00 AM,VALLE,EL CERRITO,Jueves,15:42,LA ESPERANZA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,53.0,FEMENINO,CASADO,EMPLEADO PARTICULAR,NO REPORTADO,PRIMARIA,COLOMBIA,76248000.0,1962.0,39-658,nggiso7664@gmail.com
12395,12/31/2021 12:00:00 AM,VALLE,LA CUMBRE,Jueves,19:04,CGTO BITACO E27,RURAL,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,58.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,76377000.0,1957.0,39-442,correo4352@colombia.gov.co
12396,12/31/2021 12:00:00 AM,VALLE,PALMIRA,Jueves,22:35,LORETO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,18.0,FEMENINO,SOLTERO,ESTUDIANTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76520000.0,1997.0,25-368,fktatq9245@unidatos.edu.co
12397,12/31/2021 12:00:00 AM,VALLE,ZARZAL,Jueves,14:30,URB. EL ZARZALITO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,27.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76895000.0,1988.0,62-735,gafkup6393@gmail.com


## Formatear columnas de fecha

In [382]:
# Creating a lambda expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")

In [383]:
# Applying the validation to all values in column
homicides_df["FECHA"].apply(dateparse)

0       2021-01-01 12:00:00
1       2021-01-01 12:00:00
2       2021-01-01 12:00:00
3       2021-01-01 12:00:00
4       2021-01-01 12:00:00
                ...        
12394   2021-12-31 12:00:00
12395   2021-12-31 12:00:00
12396   2021-12-31 12:00:00
12397   2021-12-31 12:00:00
12398   2021-12-31 12:00:00
Name: FECHA, Length: 12399, dtype: datetime64[ns]

In [384]:
# Creating a function for validating which value is causing the previous error
def error_in_format(x):
    try:
        datetime.strptime(x, "%m/%d/%Y %H:%M:%S %p")
        return False
    except:
        return True

In [385]:
# Using the function for validation
homicides_df.loc[homicides_df["FECHA"].apply(error_in_format)]
# THE LAST ROW IS CAUSING THE ERROR

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO


In [386]:
# Deleting a row by its index
#homicides_df.drop(12399, inplace = True)

In [387]:

# Trying to parse the datetime string again
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [388]:
# Counting homicides by hour
homicides_df["FECHA"].dt.hour.value_counts()

# All datetime hour parts are the same

12    12399
Name: FECHA, dtype: int64

In [389]:
# The homicide hour is available in a different column!
# Merging both columns
homicides_df["FECHA"] = homicides_df["FECHA"].astype(str).apply(lambda x: x[:11]) + homicides_df["HORA"]

In [390]:
del homicides_df["HORA"]

In [391]:
homicides_df["FECHA"].head()

0     2021-01-01 6:00
1     2021-01-01 9:00
2    2021-01-01 19:00
3    2021-01-01 11:20
4    2021-01-01 15:00
Name: FECHA, dtype: object

In [392]:
# Making a new expression for datetime parsing
dateparse = lambda x: datetime.strptime(x, "%Y-%m-%d %H:%M")

In [393]:
# Applying the expression
homicides_df["FECHA"] = homicides_df["FECHA"].apply(dateparse)

In [394]:
homicides_df.dtypes

FECHA                datetime64[ns]
DEPARTAMENTO                 object
MUNICIPIO                    object
DIA                          object
BARRIO                       object
ZONA                         object
CLASE DE SITIO               object
ARMA O MEDIO                 object
MOVIL VICTIMA                object
MOVIL AGRESOR                object
EDAD                        float64
GENERO                       object
ESTADO CIVIL                 object
CLASE EMPLEADO               object
PROFESION                    object
ESCOLARIDAD                  object
PAIS NACE                    object
CODIGO DANE                 float64
AÑO DE NACIMIENTO           float64
CÉDULA                       object
CORREO                       object
dtype: object

In [395]:
homicides_df

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
0,2021-01-01 06:00:00,ANTIOQUIA,AMAGÁ,Jueves,EL VOLCAN,RURAL,TIENDA,ARMA BLANCA,A PIE,A PIE,44.0,MASCULINO,CASADO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,5030000.0,1971.0,42-908,agbnqg2122@unidatos.edu.co
1,2021-01-01 09:00:00,ANTIOQUIA,BARBOSA,Jueves,VDA. MATASANOS,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,30.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,NO REPORTADO,5079000.0,1985.0,15-183,rbkeui3584@gmail.com
2,2021-01-01 19:00:00,ANTIOQUIA,EL BAGRE,Jueves,PUERTO CLAVER,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,33.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5250000.0,1982.0,84-786,aorkhf9155@unidatos.edu.co
3,2021-01-01 11:20:00,ANTIOQUIA,JARDÍN,Jueves,CRISTIANIA,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,40.0,MASCULINO,CASADO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5364000.0,1975.0,31-289,dhtemr6623@unidatos.edu.co
4,2021-01-01 15:00:00,ANTIOQUIA,MEDELLÍN (CT),Juees,PICACHITO CNO REPORTADO6,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,CONTUNDENTES,A PIE,A PIE,66.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1949.0,66-363,artatj9268@unidatos.edu.co
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12394,2021-12-31 15:42:00,VALLE,EL CERRITO,Jueves,LA ESPERANZA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,53.0,FEMENINO,CASADO,EMPLEADO PARTICULAR,NO REPORTADO,PRIMARIA,COLOMBIA,76248000.0,1962.0,39-658,nggiso7664@gmail.com
12395,2021-12-31 19:04:00,VALLE,LA CUMBRE,Jueves,CGTO BITACO E27,RURAL,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,58.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,76377000.0,1957.0,39-442,correo4352@colombia.gov.co
12396,2021-12-31 22:35:00,VALLE,PALMIRA,Jueves,LORETO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,18.0,FEMENINO,SOLTERO,ESTUDIANTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76520000.0,1997.0,25-368,fktatq9245@unidatos.edu.co
12397,2021-12-31 14:30:00,VALLE,ZARZAL,Jueves,URB. EL ZARZALITO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,27.0,MASCULINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76895000.0,1988.0,62-735,gafkup6393@gmail.com


## Estandarizar los Nombres de departamentos


In [61]:
# Creating a dictionary representing the valid departments for Colombia
departments_list = ['ANTIOQUIA', 'ATLÁNTICO', 'BOLÍVAR', 'BOYACÁ', 'CALDAS', 'CAQUETÁ',
       'CASANARE', 'CAUCA', 'CESAR', 'CHOCÓ', 'CÓRDOBA', 'META',
       'CUNDINAMARCA', 'HUILA', 'MAGDALENA', 'NARIÑO', 'PUTUMAYO',
       'RISARALDA', 'SANTANDER', 'SUCRE', 'TOLIMA', 'VALLE',
       'NORTE DE SANTANDER', 'GUAJIRA', 'QUINDÍO', 'SAN ANDRÉS Y PROVIDENCIA', 'ARAUCA',
       'GUAINÍA', 'VICHADA', 'VAUPÉS', 'GUAVIARE', 'AMAZONAS']

In [62]:
# Finding values not matching with the dictionary
homicides_df.loc[~homicides_df["DEPARTAMENTO"].isin(departments_list), "DEPARTAMENTO"].unique()

array(['SAN ANDRÉS', 'N. DE SANTANDER'], dtype=object)

<span style="color:red">TO DO: Replace the values identified as error to a valid value from the dictionary</span>

<span style="color:red">Hint: You can use the replace() pandas function</span>

In [63]:
homicides_df["DEPARTAMENTO"].replace({"N. DE SANTANDER": "NORTE DE SANTANDER", "SAN ANDRÉS": "SAN ANDRÉS Y PROVIDENCIA"}, inplace=True)

In [64]:
homicides_df.loc[~homicides_df["DEPARTAMENTO"].isin(departments_list), "DEPARTAMENTO"].unique()

array([], dtype=object)

## Drop para las cedulas duplicadas y arreglar formatos


In [65]:
# Detecting duplicates by "CÉDULA" column
duplicates_by_cedula = homicides_df.loc[homicides_df["CÉDULA"].duplicated(keep = False)]

In [66]:
duplicates_by_cedula.shape

(1627, 21)

In [67]:
# Showing some examples
duplicates_by_cedula.sort_values("CÉDULA", ascending = True).head(3)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
4041,2021-05-03 23:30:00,CAUCA,EL TAMBO,Domingo,LA VICTORIA,RURAL,"BARES, CANTINAS Y SIMILARES",ARMA DE FUEGO,A PIE,A PIE,45.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,19256000.0,1970.0,10-048,aitufn1227@gmail.com
3810,2021-04-26 03:00:00,ANTIOQUIA,SAN JERÓNIMO,Domingo,LA PLAYA,URBANA,"HOTELES, RESIDENCIAS, Y SIMILARES.",CUERDA/SOGA/CADENA,A PIE,A PIE,25.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5656000.0,1990.0,10-048,ibbcpu2509@unidatos.edu.co
693,2021-01-19 05:30:00,ANTIOQUIA,SALGAR,Lunes,LA HABANA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,58.0,MASCULINO,SOLTERO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5642000.0,57.0,10-079,unhoqj1172@unidatos.edu.co


<span style="color:red">TODO: Delete records with "CÉDULA" duplicated</span>

<span style="color:red">Hint: You can use drop_duplicates() pandas function.</span>

In [68]:
homicides_df=homicides_df.drop_duplicates('CÉDULA')

In [69]:
duplicates_by_cedula = homicides_df.loc[homicides_df["CÉDULA"].duplicated(keep = False)]

In [70]:
duplicates_by_cedula.shape

(0, 21)

In [71]:
# Using regular expressions for validating if "CÉDULA" values match the pattern XX-XXX
cedula_malformed = homicides_df.loc[homicides_df["CÉDULA"].apply(lambda x: (re.match("\d{2}-\d{3}", x) is None))]

In [72]:
cedula_malformed.shape

(28, 21)

In [74]:
cedula_malformed.head(4)

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
172,2021-01-03 11:00:00,META,MESETAS,Sábado,VEREDA EL CAFRE,RURAL,ZONA SELVÁTICA,MINA ANTIPERSONA,A PIE,A PIE,24.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,50330000.0,1991.0,680-21,oohghd8899@gmail.com
1114,2021-02-01 12:30:00,ATLÁNTICO,BARRANQUILLA (CT),Domingo,LA LUZ,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,29.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,8001000.0,1986.0,140-17,correo5853@colombia.gov.co
2119,2021-03-06 15:30:00,VALLE,CALI (CT),Viernes,QUINTAS DEL SOL E14,URBANA,DENTRO DE LA VIVIENDA,ARMA BLANCA,A PIE,A PIE,20.0,FEMENINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1995.0,975-31,ohbqrk3631@unidatos.edu.co
3309,2021-04-11 10:30:00,CAQUETÁ,FLORENCIA (CT),Sávado,VIA MORELIA,RURAL,CARCELES,CORTANTES,A PIE,A PIE,23.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,18001000.0,1992.0,348-66,pdkqur8407@unidatos.edu.co


<span style="color:red">TODO: Fix the malformed "CÉDULA" values.</span>

In [96]:
homicides_df['CÉDULA']=homicides_df["CÉDULA"].str.replace('-','')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [97]:
homicides_df['CÉDULA']

0        42908
1        15183
2        84786
3        31289
4        66363
         ...  
12394    39658
12395    39442
12396    25368
12397    62735
12398    47126
Name: CÉDULA, Length: 11563, dtype: object

## <span style="color:red">TODO: Make something similar to check and fix the "CORREO" column (PATTERN: 4 digits before the @, only .edu.co and .com domains are allowed)</span>

In [512]:
email_sinCom = homicides_df.loc[homicides_df["CORREO"].apply(lambda x: (re.search("[a-z0-9]+@[a-z0-9]+.com", x) is None))]

aca encontramos todos los que no tienen .com

In [513]:
email_sinCom

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
0,01/01/2021 12:00:00 AM,ANTIOQUIA,AMAGÁ,Jueves,6:00,EL VOLCAN,RURAL,TIENDA,ARMA BLANCA,A PIE,A PIE,44.0,MASCULINO,CASADO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,5030000.0,1978.0,42-908,agbnqg2122@unidatos.edu.co
2,01/01/2021 12:00:00 AM,ANTIOQUIA,EL BAGRE,Jueves,19:00,PUERTO CLAVER,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,33.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5250000.0,1989.0,84-786,aorkhf9155@unidatos.edu.co
3,01/01/2021 12:00:00 AM,ANTIOQUIA,JARDÍN,Jueves,11:20,CRISTIANIA,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,40.0,MASCULINO,CASADO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5364000.0,1982.0,31-289,dhtemr6623@unidatos.edu.co
4,01/01/2021 12:00:00 AM,ANTIOQUIA,MEDELLÍN (CT),Jueves,15:00,PICACHITO CNO REPORTADO6,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,CONTUNDENTES,A PIE,A PIE,66.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1956.0,66-363,artatj9268@unidatos.edu.co
5,01/01/2021 12:00:00 AM,ANTIOQUIA,MEDELLÍN (CT),Jueves,0:47,CORDOBA CNO REPORTADO7,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,42.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5001000.0,1980.0,54-450,grlcsj8696@unidatos.edu.co
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12392,12/31/2021 12:00:00 AM,VALLE,CALI (CT),Jueves,11:30,BRISAS DE LOS ALAMOS E2,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,62.0,MASCULINO,CASADO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1960.0,91-774,correo295@unidatos.edu.co
12393,12/31/2021 12:00:00 AM,VALLE,CALI (CT),Jueves,23:30,LAS ORQUIDEAS E14,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,43.0,MASCULINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1979.0,36-531,cqlqhl7009@unidatos.edu.co
12395,12/31/2021 12:00:00 AM,VALLE,LA CUMBRE,Jueves,19:04,CGTO BITACO E27,RURAL,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,58.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,76377000.0,1964.0,39-442,correo4352@colombia.gov.co
12396,12/31/2021 12:00:00 AM,VALLE,PALMIRA,Jueves,22:35,LORETO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,18.0,FEMENINO,SOLTERO,ESTUDIANTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76520000.0,2004.0,25-368,fktatq9245@unidatos.edu.co


In [514]:
email_malformed = email_sinCom.loc[email_sinCom["CORREO"].apply(lambda x: (re.match("[a-z0-9]+@[a-z0-9]+.edu.co", x) is None))]

aca quitamos los que no sean .edu.co, la siguiente lista, son los correos qu no deben estar en nuestro data set, por lo cual si lo negamos obtenemos el datset con lo que necesitamos. 

In [515]:
email_malformed

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
36,01/01/2021 12:00:00 AM,CAUCA,PATÍA,Jueves,8:00,C/MIENTO PIEDRASENTADA,RURAL,CARRETERA,ARMA DE FUEGO,VEHICULO,A PIE,26.0,MASCULINO,SOLTERO,EMPLEADO EJERCITO,NO REPORTADO,SECUNDARIA,COLOMBIA,19532000.0,1996.0,69-084,correo4714@colombia.gov.co
47,01/01/2021 12:00:00 AM,CHOCÓ,TADÓ,Jueves,9:00,POPULAR,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,34.0,MASCULINO,CASADO,EMPLEADO POLICIAL,POLICIA,TECNICO,COLOMBIA,27787000.0,1988.0,29-957,correo8297@colombia.gov.co
55,01/01/2021 12:00:00 AM,CUNDINAMARCA,BOGOTÁ D.C. (CT),Jueves,1:30,ÁLVARO BERNAL SEGURA E-19,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,25.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,SECUNDARIA,COLOMBIA,11001000.0,1997.0,35-260,correo7285@colombia.gov.co
84,01/01/2021 12:00:00 AM,NARIÑO,SAN ANDRES DE TUMACO,Jueves,8:40,PANAMA,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,26.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,52835000.0,1996.0,55-611,correo8342@colombia.gov.co
104,01/01/2021 12:00:00 AM,VALLE,CALI (CT),Jueves,3:40,12 DE OCTUBRE E12,URBANA,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,49.0,MASCULINO,CASADO,EMPLEADO PUBLICO,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1973.0,87-287,correo6479@colombia.gov.co
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12370,12/31/2021 12:00:00 AM,CÓRDOBA,VALENCIA,Jueves,21:30,CGTO VILLANUEVA,RURAL,FINCAS Y SIMILARES,ARMA DE FUEGO,A PIE,PASAJERO MOTOCICLETA,19.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,23855000.0,2003.0,10-838,correo7745@colombia.gov.co
12378,12/31/2021 12:00:00 AM,CUNDINAMARCA,BOGOTÁ D.C. (CT),Jueves,23:50,ATAHUALPA ENO REPORTADO9,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,16.0,MASCULINO,SOLTERO,DESEMPLEADO,NO REPORTADO,SECUNDARIA,COLOMBIA,11001000.0,2006.0,49-948,correo9786@colombia.gov.co
12380,12/31/2021 12:00:00 AM,CUNDINAMARCA,SESQUILÉ,Jueves,22:00,VER. NESCUATA,RURAL,TIENDA,ARMA BLANCA,A PIE,A PIE,49.0,MASCULINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,25736000.0,1973.0,49-127,correo8859@colombia.gov.co
12387,12/31/2021 12:00:00 AM,SANTANDER,GUACA,Jueves,4:18,VRDA SISOTA ALTO,RURAL,TIENDA,ARMA BLANCA,A PIE,A PIE,56.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,68318000.0,1966.0,76-263,correo3558@colombia.gov.co


In [516]:
email_ok = email_sinCom.loc[~email_sinCom["CORREO"].apply(lambda x: (re.match("[a-z0-9]+@[a-z0-9]+.edu.co", x) is None))]

In [517]:
email_ok

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
0,01/01/2021 12:00:00 AM,ANTIOQUIA,AMAGÁ,Jueves,6:00,EL VOLCAN,RURAL,TIENDA,ARMA BLANCA,A PIE,A PIE,44.0,MASCULINO,CASADO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,5030000.0,1978.0,42-908,agbnqg2122@unidatos.edu.co
2,01/01/2021 12:00:00 AM,ANTIOQUIA,EL BAGRE,Jueves,19:00,PUERTO CLAVER,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,33.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5250000.0,1989.0,84-786,aorkhf9155@unidatos.edu.co
3,01/01/2021 12:00:00 AM,ANTIOQUIA,JARDÍN,Jueves,11:20,CRISTIANIA,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,40.0,MASCULINO,CASADO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5364000.0,1982.0,31-289,dhtemr6623@unidatos.edu.co
4,01/01/2021 12:00:00 AM,ANTIOQUIA,MEDELLÍN (CT),Jueves,15:00,PICACHITO CNO REPORTADO6,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,CONTUNDENTES,A PIE,A PIE,66.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1956.0,66-363,artatj9268@unidatos.edu.co
5,01/01/2021 12:00:00 AM,ANTIOQUIA,MEDELLÍN (CT),Jueves,0:47,CORDOBA CNO REPORTADO7,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,42.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5001000.0,1980.0,54-450,grlcsj8696@unidatos.edu.co
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12391,12/31/2021 12:00:00 AM,VALLE,CALI (CT),Jueves,0:05,TERRON COLORADO E1,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,23.0,MASCULINO,SOLTERO,ESTUDIANTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1999.0,61-346,bbnsnk4611@unidatos.edu.co
12392,12/31/2021 12:00:00 AM,VALLE,CALI (CT),Jueves,11:30,BRISAS DE LOS ALAMOS E2,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,62.0,MASCULINO,CASADO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1960.0,91-774,correo295@unidatos.edu.co
12393,12/31/2021 12:00:00 AM,VALLE,CALI (CT),Jueves,23:30,LAS ORQUIDEAS E14,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,43.0,MASCULINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1979.0,36-531,cqlqhl7009@unidatos.edu.co
12396,12/31/2021 12:00:00 AM,VALLE,PALMIRA,Jueves,22:35,LORETO,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,18.0,FEMENINO,SOLTERO,ESTUDIANTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76520000.0,2004.0,25-368,fktatq9245@unidatos.edu.co


acá quitamos los que no tiene 4 digitos

In [518]:
email_sin4 = email_ok.loc[email_ok["CORREO"].apply(lambda x: (re.search("\d{4}@[a-z]+\.[a-z]{2,3}", x) is None))]

In [519]:
email_sin4

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
28,01/01/2021 12:00:00 AM,CAQUETÁ,SAN JOSÉ DEL FRAGUA,Jueves,15:00,CENTRO,URBANA,DISCOTECAS,ARMA DE FUEGO,A PIE,A PIE,24.0,MASCULINO,UNION LIBRE,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,18610000.0,1998.0,94-565,correo975@unidatos.edu.co
60,01/01/2021 12:00:00 AM,CUNDINAMARCA,BOGOTÁ D.C. (CT),Jueves,9:50,LAS COLINAS E-18,URBANA,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,37.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,11001000.0,1985.0,96-322,correo870@unidatos.edu.co
63,01/01/2021 12:00:00 AM,CUNDINAMARCA,BOGOTÁ D.C. (CT),Jueves,17:53,LA ESTRELLA SECTOR LAGOS E-19,URBANA,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,19.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,11001000.0,2003.0,45-426,correo102@unidatos.edu.co
79,01/01/2021 12:00:00 AM,NARIÑO,GUAITARILLA,Jueves,16:00,VEREDA AHUMADA CHIQUITA,RURAL,VIAS PUBLICAS,ARMA BLANCA,A PIE,A PIE,26.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,52320000.0,1996.0,54-950,correo912@unidatos.edu.co
85,01/01/2021 12:00:00 AM,PUTUMAYO,VALLE DEL GUAMUEZ,Jueves,8:11,EL EDEN,URBANA,TERMINAL DE TRANSPORTES,ARMA BLANCA,A PIE,A PIE,28.0,MASCULINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,PRIMARIA,COLOMBIA,86865000.0,1994.0,37-347,correo670@unidatos.edu.co
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12346,12/30/2021 12:00:00 AM,VALLE,TULUÁ,Miércoles,6:45,Urbanizacion Buenos Aires,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,24.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76834000.0,1998.0,50-889,correo650@unidatos.edu.co
12354,12/31/2021 12:00:00 AM,ANTIOQUIA,MEDELLÍN (CT),Jueves,0:46,GIRARDOT CNO REPORTADO5,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,34.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5001000.0,1988.0,74-042,correo183@unidatos.edu.co
12386,12/31/2021 12:00:00 AM,NORTE DE SANTANDER,VILLA DEL ROSARIO,Jueves,15:20,BARRIO MONTEVIDEO,URBANA,DENTRO DE LA VIVIENDA,ARMA DE FUEGO,A PIE,A PIE,49.0,MASCULINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,PRIMARIA,COLOMBIA,54874000.0,1973.0,87-806,correo361@unidatos.edu.co
12392,12/31/2021 12:00:00 AM,VALLE,CALI (CT),Jueves,11:30,BRISAS DE LOS ALAMOS E2,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,62.0,MASCULINO,CASADO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1960.0,91-774,correo295@unidatos.edu.co


Por lo cual si lo negamos, obtenemos los datos que queremos

In [520]:
email_ok2 = email_ok.loc[~email_ok["CORREO"].apply(lambda x: (re.search("\d{4}@[a-z]+\.[a-z]{2,3}", x) is None))]

In [521]:
email_ok2

,FECHA,DEPARTAMENTO,MUNICIPIO,DIA,HORA,BARRIO,ZONA,CLASE DE SITIO,ARMA O MEDIO,MOVIL VICTIMA,MOVIL AGRESOR,EDAD,GENERO,ESTADO CIVIL,CLASE EMPLEADO,PROFESION,ESCOLARIDAD,PAIS NACE,CODIGO DANE,AÑO DE NACIMIENTO,CÉDULA,CORREO
0,01/01/2021 12:00:00 AM,ANTIOQUIA,AMAGÁ,Jueves,6:00,EL VOLCAN,RURAL,TIENDA,ARMA BLANCA,A PIE,A PIE,44.0,MASCULINO,CASADO,INDEPENDIENTE,NO REPORTADO,PRIMARIA,COLOMBIA,5030000.0,1978.0,42-908,agbnqg2122@unidatos.edu.co
2,01/01/2021 12:00:00 AM,ANTIOQUIA,EL BAGRE,Jueves,19:00,PUERTO CLAVER,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,33.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5250000.0,1989.0,84-786,aorkhf9155@unidatos.edu.co
3,01/01/2021 12:00:00 AM,ANTIOQUIA,JARDÍN,Jueves,11:20,CRISTIANIA,RURAL,FINCAS Y SIMILARES,ARMA BLANCA,A PIE,A PIE,40.0,MASCULINO,CASADO,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,5364000.0,1982.0,31-289,dhtemr6623@unidatos.edu.co
4,01/01/2021 12:00:00 AM,ANTIOQUIA,MEDELLÍN (CT),Jueves,15:00,PICACHITO CNO REPORTADO6,URBANA,FRENTE A RESIDENCIAS - VIA PUBLICA,CONTUNDENTES,A PIE,A PIE,66.0,MASCULINO,UNION LIBRE,DESEMPLEADO,NO REPORTADO,PRIMARIA,COLOMBIA,5001000.0,1956.0,66-363,artatj9268@unidatos.edu.co
5,01/01/2021 12:00:00 AM,ANTIOQUIA,MEDELLÍN (CT),Jueves,0:47,CORDOBA CNO REPORTADO7,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,42.0,MASCULINO,SOLTERO,INDEPENDIENTE,NO REPORTADO,SECUNDARIA,COLOMBIA,5001000.0,1980.0,54-450,grlcsj8696@unidatos.edu.co
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12388,12/31/2021 12:00:00 AM,TOLIMA,IBAGUÉ (CT),Jueves,23:00,VRDA. LA CASCADA,RURAL,"BARES, CANTINAS Y SIMILARES",ARMA DE FUEGO,A PIE,A PIE,27.0,MASCULINO,UNION LIBRE,AGRICULTOR,NO REPORTADO,PRIMARIA,COLOMBIA,73001000.0,1995.0,98-474,itcmlr3548@unidatos.edu.co
12389,12/31/2021 12:00:00 AM,VALLE,CALI (CT),Jueves,4:45,LOS COMUNEROS II E13,URBANA,DENTRO DE LA VIVIENDA,ARMA BLANCA,A PIE,A PIE,32.0,FEMENINO,SOLTERO,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,NO REPORTADO,76001000.0,1990.0,59-800,umptof1764@unidatos.edu.co
12391,12/31/2021 12:00:00 AM,VALLE,CALI (CT),Jueves,0:05,TERRON COLORADO E1,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,23.0,MASCULINO,SOLTERO,ESTUDIANTE,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1999.0,61-346,bbnsnk4611@unidatos.edu.co
12393,12/31/2021 12:00:00 AM,VALLE,CALI (CT),Jueves,23:30,LAS ORQUIDEAS E14,URBANA,VIAS PUBLICAS,ARMA DE FUEGO,A PIE,A PIE,43.0,MASCULINO,UNION LIBRE,EMPLEADO PARTICULAR,NO REPORTADO,SECUNDARIA,COLOMBIA,76001000.0,1979.0,36-531,cqlqhl7009@unidatos.edu.co


## Punto arreglar dia

In [420]:
# Showing different values for "DIA" column
homicides_df["DIA"].unique()

array(['Jueves', 'Juees', 'Jueces', 'juves', 'Juevrs', 'Viernes',
       'Viermes', 'iernes', 'virnes', 'Vierens', 'Sábado', 'Sabadi',
       'Sabado', 'sábad', 'Sávado', 'Ssbado', 'Domingo', 'Domungo',
       'Doningo', 'domungo', 'Lunes', 'lune', 'Luns', 'Lumes', 'kunes',
       'Lnues', 'Martes', 'Mates', 'Marte', 'mates', 'Miércoles',
       'Miwrcoles', 'Mircoles', 'Voernes', 'domnigo', 'Maryes',
       'Miercoles', 'miércles', 'Dominog', 'Msrtes', 'Mirrcoles'],
      dtype=object)

In [421]:
# Calculating the distance between two words using the Levenshtein method
pylev.levenshtein('sábado', 'sabaod')

3

In [422]:
pylev.levenshtein('sábado', 'viernes')

7

<span style="color:red">TODO: Create a function to fix the digitation errors for "DIA" column.</span>

In [423]:
days=['Lunes','Martes','Miércoles','Jueves','Viernes','Sábado','Domingo']
bad_days=homicides_df["DIA"].unique()

array(['Jueves', 'Juees', 'Jueces', 'juves', 'Juevrs', 'Viernes',
       'Viermes', 'iernes', 'virnes', 'Vierens', 'Sábado', 'Sabadi',
       'Sabado', 'sábad', 'Sávado', 'Ssbado', 'Domingo', 'Domungo',
       'Doningo', 'domungo', 'Lunes', 'lune', 'Luns', 'Lumes', 'kunes',
       'Lnues', 'Martes', 'Mates', 'Marte', 'mates', 'Miércoles',
       'Miwrcoles', 'Mircoles', 'Voernes', 'domnigo', 'Maryes',
       'Miercoles', 'miércles', 'Dominog', 'Msrtes', 'Mirrcoles'],
      dtype=object)

In [496]:
homicides_df["DIA"]

0        Jueves
1        Jueves
2        Jueves
3        Jueves
4         Juees
          ...  
12394    Jueves
12395    Jueves
12396    Jueves
12397    Jueves
12398    Jueves
Name: DIA, Length: 12399, dtype: object

In [497]:
def fix_dia(df1):
  df=df1
  for day in df1["DIA"]:
    less=5
    best_day=day
    for y in days:
      distance =pylev.levenshtein(day,y)
      if (distance == 0):
        #print ('igual',day,y)
        best_day=y
        break
      else:
        if (distance < less):
          less = distance
          best_day =y 
          #print(best_day,day)
    df["DIA"] = df["DIA"].replace(day,best_day)
      
    #df.loc[df["DIA"].replace(day,best_day)]
  return ('termine')
          

In [498]:
fix_dia(homicides_df)

'termine'

In [499]:
homicides_df["DIA"].unique()

array(['Jueves', 'Lunes', 'Viernes', 'Sábado', 'Domingo', 'Martes',
       'Miércoles'], dtype=object)

## Arreglar nacimiento

In [340]:
homicides_df[["AÑO DE NACIMIENTO", "EDAD"]].head(10)

,AÑO DE NACIMIENTO,EDAD
0,1971.0,44.0
1,1985.0,30.0
2,1982.0,33.0
3,1975.0,40.0
4,1949.0,66.0
5,1973.0,42.0
6,1990.0,25.0
7,1978.0,37.0
8,1990.0,25.0
9,1985.0,30.0


<span style="color:red">TODO: Fix the "AÑO DE NACIMIENTO" column using the column "EDAD".</span>

In [500]:
import datetime

today = datetime.date.today()

year = today.year

In [501]:
homicides_df["AÑO DE NACIMIENTO"]=year-homicides_df["EDAD"]

In [502]:
homicides_df[["AÑO DE NACIMIENTO", "EDAD"]].head(10)

,AÑO DE NACIMIENTO,EDAD
0,1978.0,44.0
1,1992.0,30.0
2,1989.0,33.0
3,1982.0,40.0
4,1956.0,66.0
5,1980.0,42.0
6,1997.0,25.0
7,1985.0,37.0
8,1997.0,25.0
9,1992.0,30.0
